In [1]:
import pandas as pd
import numpy as np

In [2]:
cbcl = pd.read_csv('cbcl_1_5-2023-07-21.csv')
iq = pd.read_csv("iq-2023-07-21.csv")

basic = pd.read_csv("basic_medical_screening-2023-07-21.csv")
dcdq = pd.read_csv("dcdq-2023-07-21.csv")
vineland = pd.read_csv("vineland-3-2023-07-21.csv")
rbsr = pd.read_csv("rbsr-2023-07-21.csv")
scq = pd.read_csv("scq-2023-07-21.csv")

background = pd.read_csv("background_history_child-2023-07-21.csv")
core = pd.read_csv("core_descriptive_variables-2023-07-21.csv")
indi = pd.read_csv("individuals_registration-2023-07-21.csv")

C:\Users\DELL\AppData\Local\Temp\ipykernel_5404\3082356719.py:5: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  dcdq = pd.read_csv("dcdq-2023-07-21.csv")
C:\Users\DELL\AppData\Local\Temp\ipykernel_5404\3082356719.py:11: DtypeWarning: Columns (22,24) have mixed types. Specify dtype option on import or set low_memory=False.
  core = pd.read_csv("core_descriptive_variables-2023-07-21.csv")
C:\Users\DELL\AppData\Local\Temp\ipykernel_5404\3082356719.py:12: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  indi = pd.read_csv("individuals_registration-2023-07-21.csv")


In [18]:
# modified code to include all parameters for calculating cognitive impairment
# dfs = [iq, vineland, basic, cbcl, core, background, rbsr, dcdq, indi, scq]
dfs = [basic, indi, core, scq, background, rbsr, dcdq, vineland, cbcl, iq]  # SORTED in descending order
total = dfs[0].copy(deep=True)

suffixes = ['_' + str(i) for i in range(1, len(dfs))]

for df, sfx in zip(dfs[1:], suffixes):
    total = pd.merge(total, df, on='subject_sp_id', how="left", suffixes=(sfx, ''))

In [35]:
less5 = total[total['age_at_eval_months_3'] < 61]

In [34]:
less5.shape

(27088, 772)

In [39]:
def check_cognitive_impairment(row):
    if row['invalid'] != 1:
        if pd.notnull(row['fsiq_score']) or pd.notnull(row['nviq_score']):
            if pd.notnull(row['fsiq_score']) and row['fsiq_score'] < 80:
                return 1
            elif pd.notnull(row['nviq_score']) and row['nviq_score'] < 80:
                return 1
            else:
                return 0
        elif pd.notnull(row['fsiq_ratio']) or pd.notnull(row['nviq_ratio']):
            if pd.notnull(row['fsiq_ratio']) and row['fsiq_ratio'] < 80:
                return 1
            elif pd.notnull(row['nviq_ratio']) and row['nviq_ratio'] < 80:
                return 1
            else:
                return 0
        elif pd.notnull(row['intel_class']):
            if row['intel_class'] in ['profound', 'severe', 'moderate', 'mild', 'border', 'id_nos']:
                return 1
            else:
                return 0
        elif pd.notnull(row['communication_standard']):
            if row['communication_standard'] < 80:
                return 1
            else:
                return 0
        elif pd.notnull(row['cognitive_impairment_latest']):
            if (row['cognitive_impairment_latest'] == 1):
                return 1
            else:
                return 0
        elif pd.notnull(row['age_at_registration_years']) and pd.notnull(row['language_level_latest']):
            if (row['age_at_registration_years'] >= 4) and (row['language_level_latest'] != 'Uses longer sentences of his/her own and is able to tell you something that happened'):
                return 1
            else:
                return 0
        elif pd.notnull(row['cog_age_level']):
            if row['cog_age_level'] == 'signif_below_age' : 
                return 1
            else:
                return 0
        elif pd.notnull(row['cog_test_score']):
            if row['cog_test_score'] in ['24_below', '25_39', '40_54', '55_69', '70_79']:
                return 1
            else:
                return 0
        elif pd.notnull(row['age_at_registration_months']) and pd.notnull(row['motor_able']) and pd.notnull(row['dev_lang']):
            if (row['age_at_registration_months'] >= 60) and (row['motor_able'] != 1) and (row['dev_lang'] == 1):
                return 1
            else:
                return 0
        elif pd.notnull(row['regress_lang_return_y_n']) and pd.notnull(row['language_level_at_enrollment']):
            if (row['regress_lang_return_y_n'] == 4) and (row['language_level_at_enrollment'] != 'Uses longer sentences of his/her own and is able to tell you something that happened'):
                return 1
            else:
                return 0
        elif pd.notnull(row['age_at_eval_years']) and pd.notnull(row['cog_age_equivalent']) and pd.notnull(row['cog_age_equivalent_in_months']):
            if (row['age_at_eval_years'] < 15) and pd.notnull(row['cog_age_equivalent']) and (row['cog_age_equivalent_in_months']/12*100 < 80):
                return 1
            else:
                return 0

# Define the mapping for cog_age_equivalent
cog_age_map = {
    'under_12mos': 6,
    '12_18mos': 15,
    '18_24mos': 21,
    '2_3years': 30,
    '4_5years': 54,
    '6_8years': 84,
    '9_11years': 120,
    '12_or_older': 144
}

# Add a new column 'cog_age_equivalent_in_months' based on 'cog_age_equivalent'
less5.loc[:, 'cog_age_equivalent_in_months'] = less5['cog_age_equivalent'].apply(lambda x: cog_age_map.get(x, 0))


# create a parameter called deriv_cog_impair in less5 df
less5.loc[:, 'deriv_cog_impair'] = None

# Apply the function to each row of the DataFrame
less5.loc[:, 'deriv_cog_impair'] = less5.apply(check_cognitive_impairment, axis=1)

In [40]:
less5['deriv_cog_impair'].value_counts(dropna=False)

deriv_cog_impair
0.0    12366
NaN     8257
1.0     6465
Name: count, dtype: int64

In [38]:
less5

,subject_sp_id,respondent_sp_id_3,family_sf_id_1,biomother_sp_id_1,biofather_sp_id_1,sex_1,current_depend_adult_1,asd_1,age_at_eval_months_3,age_at_eval_years_3,...,nviq,nviq_vers,nviq_other,nviq_score,nviq_floor,fsiq_ratio,viq_ratio,nviq_ratio,cog_age_equivalent_in_months,deriv_cog_impair
28,SP0000090,SP0000088,SF0000057,SP0000088,SP0001268,Male,NaN,True,49,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0
47,SP0000081,SP0000044,SF0000081,SP0000044,SP0000554,Male,NaN,True,53,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0
106,SP0000162,SP0000129,SF0000162,SP0000129,SP0000435,Male,NaN,True,55,4,...,NaN,NaN,NaN,106.0,NaN,NaN,NaN,NaN,0,NaN
121,SP0000430,SP0000093,SF0000191,SP0000093,NaN,Female,NaN,False,54,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
126,SP0000203,SP0000169,SF0000203,SP0000169,SP0001544,Male,NaN,True,49,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192929,SP0563941,SP0563940,SF0563941,SP0563940,NaN,Male,NaN,True,43,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0
192934,SP0563964,SP0563962,SF0563963,SP0563962,NaN,Female,NaN,True,30,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0
192944,SP0564037,SP0564036,SF0564037,SP0564036,NaN,Female,NaN,True,44,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0
192955,SP0564094,SP0564092,SF0564093,SP0564092,NaN,Female,NaN,False,45,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN


In [7]:
# save CSV files
less5.to_csv("less5_2.csv", index=False)

In [43]:
ref129 = less5.copy()

In [44]:
# extract required parameters

model_para = [
    # 
    "subject_sp_id",
    "asd_5",
    "deriv_cog_impair",
    # BACKGROUND --------------
    "age_onset_mos",
    "cog_age_level",   # ENCODING
    "family_hx_asd_mat_aunt_uncle",
    "father_highest_education",
    "fed_self_spoon_age_mos",
    "function_age_level",   # ENCODING
    "hand",
    "intervention_counseling",
    "intervention_medication",
    "intervention_ot_fine_mot",
    "intervention_ot_sensory",
    "intervention_social_skills",
    "intervention_speech_language",
    "language_age_level",   # ENCODING
    "mother_highest_education",
    "onset_concern",
    "plateau_y_n",
    "regress_lang_return_y_n",
    "regress_lang_time_return",
    "regress_lang_y_n",
    "repeat_grade",
    "smiled_age_mos",
    "sped_504",
    "sped_aide",
    "sped_asd_class_ft",
    "sped_birth_to_three",
    "sped_iep_asd",
    "sped_iep_other",
    "sped_ot",
    "sped_other",
    "sped_other_class_ft",
    "sped_other_class_pt",
    "sped_preschool",
    "sped_priv_asd",
    "sped_pt",
    "sped_soc_skills",
    "sped_speech",
    "sped_y_n",
    "support_require",
    # 
    # SCQ -----------
    "q01_phrases",
    "q02_conversation",
    "q04_inappropriate_question",
    "q06_invented_words",
    "q09_expressions_appropriate",
    "q10_hand_tool",
    "q13_interests_intensity",
    "q14_senses",
    "q18_objects_carry",
    "q20_talk_friendly",
    "q21_copy_you",
    "q22_point_things",
    "q23_gestures_wanted",
    "q24_nod_head",
    "q25_shake_head",
    "q26_look_directly",
    "q28_things_interested",
    "q29_share",
    "q30_join_enjoyment",
    "q31_comfort",
    "q32_help_attention",
    "q33_range_expressions",
    "q35_make_believe",
    "q36_same_age",
    "q38_pay_attention",
    # 
    # DCDQ --------------------------
    "q02_catch_ball",
    "q05_run_fast_similar",
    "q06_plan_motor_activity",
    "q09_appropriate_tension_printing_writing",
    "q10_cuts_pictures_shapes",
    "q11_likes_sports_motors_skills",
    "q13_quick_competent_tidying_up",
    "fine_motor_handwriting",
    #
    # basic medical screening --------------
    "dev_id",
    "dev_lang_dis",
    "mood_or_anx",
    "neuro_oth_calc",
    "dev_speech",
    "med_cond_birth",
    "med_cond_neuro",
    "med_cond_visaud",
    "mood_anx",
    "mood_soc_anx",
    "attn_behav",
    "behav_adhd",
    "birth_prem",
    # 
    # individuals registration -------------
    "diagnosis_age",
    "ASD_services",
    "iep_asd",
    "cognitive_impairment_at_enrollment",   # ENCODING
    "language_level_at_enrollment",   # ENCODING
    # 
    # RBSR --------------------
    "q01_whole_body",
    "q03_hand_finger",
    "q07_hits_self_body",
    "q08_hits_self_against_object",
    "q09_hits_self_with_object",
    "q12_rubs",
    "q16_complete",
    "q18_checking",
    "q19_counting",
    "q22_touch_tap",
    "q32_insists_walking",
    "q27_play",
    "q28_communication",
    "q29_things_same_place",
    "q31_becomes_upset",
    "q34_dislikes_changes",
    "q35_insists_door",
    "q36_likes_piece_music",
    "q39_insists_time",
    "q41_strongly_attached",
    "q43_fascination_movement"
]


ref129 = ref129[model_para]

In [45]:
ref129.rename(columns={'asd_5': 'asd'}, inplace=True)

In [46]:
# perform one hot encoding of the dataframe

# Identify categorical columns
categorical_columns = ['cog_age_level', 'father_highest_education', 'function_age_level', 'hand', 'language_age_level',
                   'mother_highest_education', 'onset_concern', 'regress_lang_time_return', 'support_require',
                   'ASD_services', 'iep_asd', 'cognitive_impairment_at_enrollment', 'language_level_at_enrollment']

# One-hot encode categorical columns
ref129_encoded = pd.get_dummies(ref129, columns=categorical_columns)

In [47]:
# drop paramters

drop_para = [
    'father_highest_education_associate_degree',
    'father_highest_education_did_not_attend_high_school',
    'father_highest_education_ged_diploma',
    'father_highest_education_graduate_or_professional_degree',
    'father_highest_education_some_high_school',
    'father_highest_education_trade_school',
    'function_age_level_above_age',
    'function_age_level_at_age',
    'function_age_level_slight_below_age',
    'hand_left',
    'hand_not_able',
    'language_age_level_at_age',
    'language_age_level_slight_below_age',
    'mother_highest_education_associate_degree',
    'mother_highest_education_baccalaureate_degree',
    'mother_highest_education_did_not_attend_high_school',
    'mother_highest_education_ged_diploma',
    'mother_highest_education_some_college',
    'mother_highest_education_some_high_school',
    'mother_highest_education_trade_school',
    'onset_concern_change_loss',
    'onset_concern_something_else',
    'onset_concern_unusual_habits',
    'onset_concern_unusual_speech',
    'support_require_substantial',
    'ASD_services_False',
    'iep_asd_False',
    'cognitive_impairment_at_enrollment_False',
    'language_level_at_enrollment_No words/does not speak'
]


ref129_dropped = ref129_encoded.drop(columns=drop_para) 

In [48]:
# List of parameters to replace
parameters_to_replace = [
    'cog_age_level_above_age', 'cog_age_level_at_age', 'cog_age_level_signif_below_age',
    'cog_age_level_slight_below_age', 'father_highest_education_baccalaureate_degree',
    'father_highest_education_high_school_graduate', 'father_highest_education_some_college',
    'function_age_level_signif_below_age', 'hand_ambi', 'hand_right',
    'language_age_level_above_age', 'language_age_level_signif_below_age',
    'mother_highest_education_graduate_or_professional_degree',
    'mother_highest_education_high_school_graduate', 'onset_concern_late_speech',
    'onset_concern_late_walk', 'onset_concern_mood', 'onset_concern_social',
    'regress_lang_time_return_5_or_more_yr', 'regress_lang_time_return_6_to_12_mo',
    'regress_lang_time_return_bt_1_to_2_yr', 'regress_lang_time_return_bt_2_to_3_yr',
    'regress_lang_time_return_bt_3_to_4_yr', 'regress_lang_time_return_bt_4_to_5_yr',
    'regress_lang_time_return_less_than_6_mo', 'support_require_minimal_no_support',
    'support_require_some_support', 'support_require_very_substantial', 'ASD_services_True',
    'iep_asd_True', 'cognitive_impairment_at_enrollment_True',
    'language_level_at_enrollment_Combines 3 words together into short sentences',
    'language_level_at_enrollment_Uses longer sentences of his/her own and is able to tell you something that happened',
    'language_level_at_enrollment_Uses single words meaningfully (for example, to request)'
]

ref_copy = ref129_dropped.copy()

# Iterate through the list of parameters and perform replacement
for param in parameters_to_replace:
    # ref129_dropped[param] = ref129_dropped[param].map({TRUE: 1, FALSE: 0})
    ref_copy[param] = ref_copy[param].map({True: 1, False: 0})

In [49]:
# save CSV files
# ref129_dropped.to_csv("ref129_2.csv", index=False)
ref_copy.to_csv("ref129_latest.csv", index=False)